In [20]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

import Base: show

In [21]:
function createSubgraph(index::Int)
    noise_add      = AdditionNode("[$index] noise_add");
    noise_node     = GaussianMeanVarianceNode("[$index] noise_node");
    noise_mean     = ConstantVariable("[$index] noise_mean", 0.0, noise_node.mean);
    noise_variance = ConstantVariable("[$index] noise_variance", 1.0, noise_node.variance);
    
    tmp = RandomVariable("[$index] tmp", noise_node.value, noise_add.in2)
    yn  = ObservedVariable("[$index] yn", noise_add.out)
    
    return (yn, noise_add)
end

function createGraph(size::Int)
    gmv_0   = GaussianMeanVarianceNode("[0] gmv");
    gmv_0_m = PriorVariable("[0] gmv_m", gmv_0.mean);
    gmv_0_v = PriorVariable("[0] gmv_v", gmv_0.variance);
    
    add_c_0 = AdditionNode("[0] add_c");
    x0      = RandomVariable("[0] x", gmv_0.value, add_c_0.in1)
    c0      = ConstantVariable("[0] c", 1.0, add_c_0.in2)
    
    index = 1
    
    prev_add_c = add_c_0
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{ObservedVariable}(undef, size)
    
    while index < size
        equality_n    = EqualityIOONode("[$index] equality")
        yn, noise_add = createSubgraph(index)
        
        xn  = RandomVariable("[$index] xn", prev_add_c.out, equality_n.in1)
        xn_ = RandomVariable("[$index] xn_", equality_n.out1, noise_add.in1)
        
        add_c = AdditionNode("[$index] add_c");
        
        xn__ = RandomVariable("[$index] xn__", equality_n.out2, add_c.in1)
        cn   = ConstantVariable("[$index] cn", 1.0, add_c.in2)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_add_c = add_c
        
        index += 1
    end
    
    last_noise_add = AdditionNode("[last] noise_add");
    x_last         = RandomVariable("[last] x", prev_add_c.out, last_noise_add.in1)
    
    last_noise_node     = GaussianMeanVarianceNode("[last] noise_node");
    last_noise_mean     = ConstantVariable("[last] noise_mean", 0.0, last_noise_node.mean);
    last_noise_variance = ConstantVariable("[last] noise_variance", 200.0, last_noise_node.variance);
    
    z      = RandomVariable("[last] z", last_noise_node.value, last_noise_add.in2)
    y_last = ObservedVariable("[last] y", last_noise_add.out)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, gmv_0_m, gmv_0_v)
end

createGraph (generic function with 1 method)

In [22]:
function smoothing_replay()
    N = 600
    data = collect(1:N) + sqrt(200.0) * randn(N);
    
    xs, ys, prior_m, prior_v = createGraph(N);
    
    keep_actor = keep(AbstractMessage)
    
    subscribe!(inference(xs[1]) |> take(1), keep_actor)
    subscribe!(inference(xs[N]) |> take(1), keep_actor)
    
    update!(prior_m, 0.0)
    update!(prior_v, 1000.0)
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    for i in 2:N-1
        subscribe!(inference(xs[i]) |> take(1), keep_actor)
    end
    
    return keep_actor.values
end

smoothing_replay (generic function with 1 method)

In [23]:
@btime smoothing_replay();

  2.817 s (21403097 allocations: 675.61 MiB)


In [24]:
smoothing_replay()

600-element Array{AbstractMessage,1}:
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=1.6924875153548262, σ=0.04085868768828826))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=600.692487515354, σ=0.040858687688288306)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.692487515354826, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.692487515354826, σ=0.04085868768828825)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=4.692487515354826, σ=0.04085868768828825)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=5.692487515354826, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=6.692487515354825, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.692487515354825, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=8.692487515354827, σ=0.04085868768828826)) 
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=9.692487515354825, σ=0.040